### **Objectives**

1. 실습명 : vLLM을 이용한 효율적인 LLM 추론과 성능 벤치마킹
2. 핵심 주제:
    1. vLLM의 PagedAttention과 KV Cache 개념 이해
    2. LoRA adapter 통합 방법 (Runtime LoRA vs Merged Model)
    3. Transformers vs vLLM 성능 비교 (속도, 메모리, 처리량)
    4. Text-to-SQL task를 위한 실전 추론 시스템 구축
3. 학습 목표 :
    1. vLLM의 PagedAttention 메커니즘을 이해하고 메모리 효율성의 원리를 설명할 수 있다.
    2. LoRA adapter를 vLLM에 통합하여 추론을 수행할 수 있다.
    3. 성능 벤치마크를 통해 vLLM의 장점을 정량적으로 분석할 수 있다.
4. 학습 개념: 키워드명 :
    1. PagedAttention
    2. KV Cache
    3. vLLM
    4. LoRA adapter
    5. Performance Benchmarking
5. 학습 방향 :
  - vLLM의 핵심 기술인 PagedAttention이 어떻게 메모리를 효율적으로 관리하는지 시뮬레이터를 통해 체험합니다.
  - 실습 코드는 단계별로 구성되어 있으며, Jupyter notebook을 통해 이론과 실습을 병행합니다.
  - 성능 벤치마킹을 통해 일반 추론 방식과 vLLM의 차이를 직접 확인하고 분석합니다.

### **Prerequisites**
```
numpy==2.1.0
pandas==2.2.3
transformers==4.56.0
torch==2.9.0+cu129
accelerate==1.10.1
bitsandbytes==0.49.1
datasets==4.0.0
peft==0.17.1
vllm==0.13.0
```

## 1: vLLM 아키텍처 및 핵심 원리
        
### 1.1 도입 배경 (Why vLLM?)

기존 HuggingFace Transformers 기반 서빙 방식은 프로덕션 환경에서 다음과 같은 병목(Bottleneck)이 존재합니다.

* **메모리 단편화(Fragmentation)**: KV Cache를 연속된 메모리 공간에 할당해야 하므로, 실제 사용되지 않는 예약 메모리(Internal Fragmentation)로 인한 VRAM 낭비가 심각합니다.
* **낮은 처리량(Throughput)**: Static Batching 방식의 비효율성으로 인해 동시 접속 처리에 한계가 있습니다.

**vLLM의 솔루션:**

* **PagedAttention**: OS의 가상 메모리(Virtual Memory) 기법을 차용하여 KV Cache를 비연속적 블록으로 관리, 메모리 효율을 극대화합니다.
* **Continuous Batching**: 요청이 들어오는 즉시 동적으로 배치에 포함시켜 GPU 가동률(Utilization)을 높입니다.

### 1.2 핵심 메커니즘 1: PagedAttention

PagedAttention은 vLLM 성능 향상의 핵심으로, 운영체제의 페이징(Paging) 시스템과 동일한 원리입니다.

**Legacy 방식 (Contiguous Allocation)**:

* 요청 시 최대 토큰 길이(Max Context Length)만큼의 연속된 메모리를 미리 확보합니다.
* 생성되지 않은 미래의 토큰을 위한 공간까지 점유하므로 **메모리 낭비(Waste)**가 발생합니다.

```
Physical Memory: [Token 1...Token N | Reserved (Unused) Space]
-> 메모리 단편화 및 OOM(Out of Memory)의 주원인

```

**PagedAttention 방식 (Non-contiguous Allocation)**:

* KV Cache를 고정 크기의 '블록(Block)' 단위로 분할하여 관리합니다.
* 물리적으로 떨어진 메모리 공간이라도 **Block Table**을 통해 논리적으로 연속된 것처럼 매핑합니다.
* 필요할 때만 블록을 동적 할당하므로 메모리 낭비가 거의 없습니다(Near-zero waste).

```
Block 0 (Physical): [Token 0-3]
Block 1 (Physical): [Token 4-7]
Block Table (Logical): {Seq_A: [Block 0, Block 1, ...]}
-> 비연속적 할당 가능, 유연한 메모리 관리

```

### 1.3 핵심 메커니즘 2: KV Cache 최적화

LLM은 자기회귀(Autoregressive) 모델이므로, 이전 토큰들의 정보를 바탕으로 다음 토큰을 생성합니다.

**KV Cache 부재 시 (Redundant Computation)**:

* 매 스텝마다 이전 시점()까지의 모든 토큰에 대해 Attention 연산을 처음부터 다시 수행합니다.
* 시퀀스 길이가 길어질수록 연산량이 $O(N^2)$로 증가하여 Latency가 급격히 나빠집니다.

**KV Cache 적용 시 (Memory-Space Tradeoff)**:

* 이전 토큰들의 Key, Value 벡터를 미리 계산하여 VRAM에 캐싱해둡니다.
* 현재 시점()의 토큰만 연산하고, 과거 데이터는 캐시를 참조(Lookup)하여 연산 복잡도를 선형적으로 줄입니다.

**Critical Issue**:

* KV Cache는 모델 사이즈와 시퀀스 길이에 비례하여 매우 큰 VRAM을 점유합니다.
* 결국 **"제한된 GPU 메모리에 얼마나 많은 KV Cache를 효율적으로 구겨 넣느냐"**가 서빙 성능(Throughput)의 결정적 요소가 되며, 이것이 vLLM이 PagedAttention을 도입한 기술적 배경입니다.

In [1]:
# 필요한 라이브러리 import
import os
import time
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch 버전: 2.9.0+cu129
CUDA 사용 가능: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


### 1.4: PagedAttention 메커니즘 시뮬레이션

이론적 개념을 구체화하기 위해, Python 환경에서 **PagedAttention의 메모리 할당 로직**을 시뮬레이션해보겠습니다. 이를 통해 가상 메모리 주소가 실제 GPU 메모리에 매핑되는 과정을 코드 레벨에서 검증합니다.

### 시뮬레이터 핵심 컴포넌트 (Simulator Components)

* **Logical Memory (Virtual Address Space)**
  * 모델이 인식하는 논리적인 토큰 시퀀스입니다.
  * 사용자 입장에서는 데이터가 연속적으로 존재하는 것처럼 보입니다.


* **Physical Memory (Physical Address Space)**
  * 실제 KV Cache 데이터가 적재되는 물리적 메모리 공간(VRAM)입니다.
  * 블록(Block) 단위로 비연속적으로 할당 및 관리됩니다.


* **Block Table (Address Translation)**
  * Logical Block을 Physical Block으로 변환하는 매핑 테이블입니다.
  * OS의 페이지 테이블(Page Table)과 동일한 역할을 수행하여 메모리 참조를 중개합니다.

In [2]:
import random
from typing import List, Dict, Optional

class PhysicalBlock:
    """
    물리적인 메모리 블록 (GPU의 작은 조각)
    여기서는 편의상 하나의 블록에 4개의 토큰을 저장한다고 가정합니다 (BLOCK_SIZE = 4).
    """
    def __init__(self, block_id: int, block_size: int = 4):
        self.block_id = block_id
        self.size = block_size
        self.data = [None] * block_size  # 처음엔 비어있음
        self.filled_count = 0

    def append(self, token: str) -> bool:
        if self.filled_count < self.size:
            self.data[self.filled_count] = token
            self.filled_count += 1
            return True  # 저장 성공
        return False  # 꽉 차서 실패

    def __repr__(self):
        return f"[Block {self.block_id}]: {self.data}"

class VLLMMemoryManager:
    """
    vLLM의 핵심인 메모리 관리자 (OS 역할)
    """
    def __init__(self, total_blocks: int = 16, block_size: int = 4):
        self.block_size = block_size
        # 1. 물리적 메모리 공간 생성 (GPU VRAM)
        self.physical_memory = [PhysicalBlock(i, block_size) for i in range(total_blocks)]
        
        # 2. 사용 가능한 블록 리스트 (Free List)
        # 실제로는 물리적으로 떨어져 있어도 상관없음을 보여주기 위해 섞습니다.
        self.free_blocks = list(range(total_blocks))
        random.shuffle(self.free_blocks) 
        
        # 3. 요청별 블록 매핑 테이블 (Request ID -> List[Physical Block IDs])
        self.block_tables: Dict[str, List[int]] = {}

    def allocate_block(self) -> Optional[int]:
        """빈 물리 블록 하나를 가져옵니다."""
        if not self.free_blocks:
            return None # OOM (Out of Memory)
        return self.free_blocks.pop()

    def append_token(self, request_id: str, token: str):
        """
        핵심 로직: 토큰을 생성해서 KV Cache에 추가하는 과정
        """
        # 1. 해당 요청의 블록 테이블이 없으면 생성
        if request_id not in self.block_tables:
            print(f"--- [New Request] '{request_id}' 시작 ---")
            self.block_tables[request_id] = []

        # 2. 현재 사용 중인 마지막 물리 블록 확인
        current_block_id = None
        if self.block_tables[request_id]:
            current_block_id = self.block_tables[request_id][-1]
        
        # 3. 블록이 없거나 꽉 찼으면, 새로운 물리 블록 할당 (PagedAttention의 핵심!)
        if current_block_id is None or \
           self.physical_memory[current_block_id].filled_count >= self.block_size:
            
            new_block_id = self.allocate_block()
            if new_block_id is None:
                raise Exception("GPU Memory Full!")
            
            self.block_tables[request_id].append(new_block_id)
            current_block_id = new_block_id
            print(f"👉 [Alloc] 새로운 물리 블록 {current_block_id}번 할당됨")

        # 4. 물리 블록에 데이터 저장
        self.physical_memory[current_block_id].append(token)
        print(f"   [Write] 토큰 '{token}' -> 물리 블록 {current_block_id}번에 저장")

    def print_state(self, request_id: str):
        """현재 메모리 상태를 시각화해서 보여줍니다."""
        print(f"\n📊 [{request_id}] 의 PagedAttention 상태")
        table = self.block_tables.get(request_id, [])
        
        # 1. Logical View (사용자가 보는 문장)
        logical_text = []
        for block_id in table:
            block = self.physical_memory[block_id]
            logical_text.extend([t for t in block.data if t is not None])
        print(f"  1) 논리적 뷰 (Logical): {logical_text}")

        # 2. Block Table (매핑 정보)
        print(f"  2) 블록 테이블 (Mapping): {table}")

        # 3. Physical View (실제 저장 위치)
        print(f"  3) 물리적 뷰 (Physical):")
        for block_id in table:
            print(f"     {self.physical_memory[block_id]}")
        print("-" * 50)

print("✅ PagedAttention 시뮬레이터 클래스 준비 완료")

✅ PagedAttention 시뮬레이터 클래스 준비 완료


In [3]:
# PagedAttention 시뮬레이션 실행
# LLM이 한 토큰씩 생성하는 상황을 흉내냅니다

# 1. 매니저 초기화 (Block Size = 4)
vllm_manager = VLLMMemoryManager(total_blocks=10, block_size=4)

# 2. 문장 생성 시뮬레이션
req_id = "User_A"
tokens = ["Deep", "Learn", "ing", "is", "very", "fun", "to", "study"]

print("🚀 토큰 생성 시뮬레이션 시작\n")
for token in tokens:
    vllm_manager.append_token(req_id, token)

# 3. 최종 상태 확인
vllm_manager.print_state(req_id)

🚀 토큰 생성 시뮬레이션 시작

--- [New Request] 'User_A' 시작 ---
👉 [Alloc] 새로운 물리 블록 3번 할당됨
   [Write] 토큰 'Deep' -> 물리 블록 3번에 저장
   [Write] 토큰 'Learn' -> 물리 블록 3번에 저장
   [Write] 토큰 'ing' -> 물리 블록 3번에 저장
   [Write] 토큰 'is' -> 물리 블록 3번에 저장
👉 [Alloc] 새로운 물리 블록 1번 할당됨
   [Write] 토큰 'very' -> 물리 블록 1번에 저장
   [Write] 토큰 'fun' -> 물리 블록 1번에 저장
   [Write] 토큰 'to' -> 물리 블록 1번에 저장
   [Write] 토큰 'study' -> 물리 블록 1번에 저장

📊 [User_A] 의 PagedAttention 상태
  1) 논리적 뷰 (Logical): ['Deep', 'Learn', 'ing', 'is', 'very', 'fun', 'to', 'study']
  2) 블록 테이블 (Mapping): [3, 1]
  3) 물리적 뷰 (Physical):
     [Block 3]: ['Deep', 'Learn', 'ing', 'is']
     [Block 1]: ['very', 'fun', 'to', 'study']
--------------------------------------------------


### 실행 결과 분석 및 기술적 시사점

시뮬레이션 결과를 통해 PagedAttention 아키텍처가 메모리 효율성을 달성하는 구체적인 메커니즘을 확인할 수 있습니다. 주요 기술적 특징은 다음과 같습니다.

#### 🔑 Key Technical Insights

**1. 물리적 메모리의 비연속성 (Non-contiguous Allocation)**

* **관찰**: 논리적 시퀀스상에서는 `'is'`(Block 0의 끝)와 `'very'`(Block 1의 시작)가 연속되지만, 물리적 메모리 주소(Physical Block Index)는 불연속적으로 매핑됩니다.
* **기술적 의의**: 물리적 주소 공간의 연속성 제약을 제거함으로써, VRAM 내 산재한 유휴 메모리 조각(Fragmented chunks)을 100% 활용할 수 있게 합니다. 이는 메모리 단편화(Fragmentation) 문제를 구조적으로 해결합니다.

**2. 할당의 효율성 (On-demand Allocation)**

* **관찰**: 새로운 물리적 블록은 현재 블록이 완전히 채워진 시점(Slot full)에만 할당됩니다.
* **기술적 의의**: 기존의 정적 할당 방식에서 발생하는 '예약된 미사용 공간(Reserved but unused memory)'에 의한 낭비를 제거합니다. 문장 생성 중단 시, 불필요한 추가 메모리 할당이 발생하지 않으므로 VRAM 점유율(Footprint)을 최소화합니다.

**3. 주소 변환 매커니즘 (Address Translation Logic)**

* **관찰**: vLLM은 `Block Table`을 통해 논리적 인덱스를 물리적 주소로 변환합니다.
* **Block Index**: `Global Token Index // Block Size`
* **Block Offset**: `Global Token Index % Block Size`


* **기술적 의의**: 이 연산 과정은 $O(1)$의 시간 복잡도를 가지므로, 토큰 조회(Lookup) 시 오버헤드가 거의 발생하지 않습니다. 즉, 메모리 효율성을 얻는 대신 감수해야 할 성능 비용(Trade-off)이 무시할 수준입니다.

## 2 vLLM 기본 사용법

### 2.1 모델 로딩
vLLM의 Inference Engine(LLM Class)을 초기화하는 진입점(Entry Point)입니다. 이 단계에서 하드웨어 리소스 할당 전략과 모델 로딩 방식이 결정되므로, 시스템 성능 최적화에 있어 가장 중요한 설정입니다.

### vLLM Engine Initialization Parameters

#### 1. `tensor_parallel_size` (TP Degree)

정의: 모델의 가중치(Weights)와 연산을 몇 개의 GPU에 분산하여 처리할지 결정하는 Tensor Parallelism의 차수(Degree)입니다. 예를 들어, GPU가 4장 들어있을 경우 tensor_parallel_size=4 입니다. 현재 노트북에서는 gpu가 1개밖에 없기 때문에 tensor_parallel_size=1 고정입니다.

* Megatron-LM 스타일의 Tensor Model Parallelism을 사용합니다.
* 각 Transformer 레이어의 행렬 연산(Matrix Multiplication)을 물리적 GPU들에 수평적으로 샤딩(Sharding)합니다.
* 예: 인 경우, 행렬 를 로 쪼개어 두 GPU가 각각 연산한 후, `All-Reduce` 통신을 통해 결과를 합칩니다.


* `value=1`: 단일 GPU 모드입니다. 모델 전체가 하나의 GPU VRAM에 적재 가능할 때 사용합니다. 통신 오버헤드가 없어 Latency 측면에서 유리할 수 있습니다.
* `value > 1`: 모델 파라미터가 커서 단일 GPU에 적재가 불가능하거나(OOM), 대규모 모델의 추론 속도를 높이고자 할 때 사용합니다.
* Constraint: 반드시 머신에 장착된 물리적 GPU 개수 이하여야 하며, GPU 간 통신 대역폭(NVLink 등)이 성능의 병목이 될 수 있음을 고려해야 합니다.



#### 2. `gpu_memory_utilization` (VRAM Budgeting)

정의: vLLM 프로세스가 사용할 수 있는 최대 VRAM 비율(Fraction)을 설정합니다. (Default: 0.9) 만약, 6GB의 VRAM GPU에서 0.9를 설정하면 6 * 0.9 = 5.4GB의 gpu를 미리 할당합니다. 실제 모델의 크기가 3GB더라도 5.4GB를 차지하게 되므로 너무 높은 값을 설정하게 되면 gpu 낭비가 되고, 반면에 너무 적게 설정하면 cpu 추론으로 넘어가기 때문에 너무 낮게 설정하면 안됩니다.

* 엔진 초기화 시 Profiling Phase가 실행됩니다.
* `(전체 VRAM * 0.9) - (모델 가중치 + Activation 오버헤드)`를 계산하여, 남은 공간을 KV Cache용 블록으로 미리 할당(Pre-allocation)합니다.


* Why not 1.0?: PyTorch 런타임이나 CUDA Context가 사용할 여유 공간(Buffer)을 남겨둬야 합니다. 1.0으로 설정 시 OOM(Out of Memory)으로 프로세스가 강제 종료될 위험이 매우 높습니다.
* Performance Trade-off: 이 값을 높이면 KV Cache 공간이 늘어나 배치 크기(Batch Size)를 키울 수 있어 처리량(Throughput)이 향상됩니다. 반면, 너무 높게 잡으면 런타임 중 피크 메모리 사용 시 충돌이 발생할 수 있으므로, 0.9~0.95 사이에서 튜닝하는 것이 일반적입니다.



#### 3. `trust_remote_code` (Execution Policy)

정의: HuggingFace Hub 등 외부 소스에서 다운로드한 모델의 커스텀 Python 코드(`modeling_*.py`) 실행을 허용할지 여부입니다.

* `transformers` 라이브러리에 아직 공식 통합(Upstream)되지 않은 최신 아키텍처나, 커스텀 모델(예: MPT, Falcon 초기 버전 등)을 로드할 때 필요합니다.
* `True`로 설정 시, 모델 가중치뿐만 아니라 모델 구조를 정의하는 Python 스크립트까지 로컬 환경에서 실행합니다.


* Security Risk: 검증되지 않은 모델 저장소에서 이 옵션을 켜면 악성 코드가 실행될 수 있는 RCE(Remote Code Execution) 취약점에 노출됩니다.
* Best Practice: 검증된 공식 리포지토리(Official Org)나 사내 모델이 아닌 경우, 코드를 먼저 검토(Audit)한 후 사용하는 것이 원칙입니다. 일반적인 Llama 2, Mistral 같은 표준 아키텍처는 `False`로 설정해도 vLLM 내부 구현체로 로딩 가능합니다.


#### 4. `enforce_eager` (CUDA Graph Strategy)

**정의:** vLLM의 핵심 최적화 기술 중 하나인 **CUDA Graphs** 사용 여부를 결정합니다. `True`로 설정하면 CUDA Graphs를 끄고, PyTorch의 기본 실행 방식인 **Eager Mode**로 강제 전환합니다. (Default: `False`)

* **CUDA Graphs란?:** GPU 연산 커널(Kernel)들의 실행 순서와 메모리 할당을 미리 '그래프' 형태로 캡처(Capture)해두고, 추론 시에는 이 그래프를 통째로 실행(Replay)하는 기술입니다. 이를 통해 CPU가 GPU에 명령을 내리는 오버헤드(Kernel Launch Overhead)를 획기적으로 줄입니다.
* **Why use `True` (Disabling Graphs)?:**
    * **Debugging:** CUDA 에러가 발생했을 때 정확한 위치를 파악하기 위해 사용합니다.
    * **Memory Issue:** CUDA Graph를 캡처하고 유지하는 데에도 일정량의 VRAM이 소모됩니다. VRAM이 극한으로 부족하여 OOM이 발생할 때, 성능 저하를 감수하고 메모리를 확보하기 위해 켤 수 있습니다.
    * **Hardware Compatibility:** 일부 구형 GPU나 특정 드라이버 환경에서 CUDA Graphs가 불안정할 경우 사용합니다.


    * **Performance Impact:** `True`로 설정할 경우, CPU 오버헤드가 증가하여 특히 작은 배치 사이즈에서의 처리량(Throughput)과 지연 시간(Latency) 성능이 저하될 수 있습니다. 프로덕션 환경에서는 `False`(기본값) 유지를 권장합니다.

In [4]:
!nvidia-smi

Sun Feb  1 00:59:54 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.05              Driver Version: 576.57         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8              2W /   78W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# Step 1: vLLM 모델 초기화
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"

print("🚀 vLLM 모델 로딩 중...")
start_time = time.time()

llm = LLM(
    model=model_name,
    tensor_parallel_size=1,       # 단일 GPU 사용
    gpu_memory_utilization=0.75,   # GPU 메모리 75% 사용 (원래 모델의 크기는 1.5GB 정도)
    trust_remote_code=True,       # 커스텀 모델 코드 신뢰
    enforce_eager=True            # CUDA Graph 캡쳐 비활성화
)

load_time = time.time() - start_time
print(f"✅ 모델 로딩 완료! ({load_time:.2f}초)")

# Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"📝 Tokenizer 준비 완료")

🚀 vLLM 모델 로딩 중...
INFO 02-01 00:59:54 [utils.py:253] non-default args: {'trust_remote_code': True, 'gpu_memory_utilization': 0.75, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'HuggingFaceTB/SmolLM2-360M-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 02-01 00:59:55 [model.py:514] Resolved architecture: LlamaForCausalLM
INFO 02-01 00:59:55 [model.py:1661] Using max model len 8192
INFO 02-01 00:59:57 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 02-01 00:59:57 [vllm.py:622] Enforce eager set, overriding optimization level to -O0
INFO 02-01 00:59:57 [vllm.py:722] Cudagraph is disabled under eager mode
WARNING 02-01 00:59:58 [interface.py:465] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
WARNING 02-01 00:59:58 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:03 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='HuggingFaceTB/SmolLM2-360M-Instruct', speculative_config=None, tokeniz

(EngineCore_DP0 pid=4930) /home/ssafy/skeleton-4/.venv/lib/python3.13/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=4930) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=4930)   warnings.warn(


(EngineCore_DP0 pid=4930) INFO 02-01 01:00:06 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:08 [weight_utils.py:527] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.69it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.69it/s]
(EngineCore_DP0 pid=4930) 


(EngineCore_DP0 pid=4930) INFO 02-01 01:00:08 [default_loader.py:308] Loading weights took 0.23 seconds
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:08 [gpu_model_runner.py:3659] Model loading took 0.6750 GiB memory and 3.259518 seconds
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:10 [gpu_worker.py:375] Available KV cache memory: 3.33 GiB
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:10 [kv_cache_utils.py:1291] GPU KV cache size: 87,168 tokens
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:10 [kv_cache_utils.py:1296] Maximum concurrency for 8,192 tokens per request: 10.64x
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:10 [core.py:259] init engine (profile, create kv cache, warmup model) took 1.83 seconds
(EngineCore_DP0 pid=4930) WARNING 02-01 01:00:11 [vllm.py:629] Inductor compilation was disabled by user settings,Optimizations settings that are only active duringInductor compilation will be ignored.
(EngineCore_DP0 pid=4930) INFO 02-01 01:00:11 [vllm.py:722] Cudagraph is disabled under eager mode

In [6]:
!nvidia-smi

Sun Feb  1 01:00:12 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.05              Driver Version: 576.57         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   52C    P4             22W /   78W |    4805MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 2.2 간단한 추론 실습

아래 코드는 LLM의 Decoding Strategy(디코딩 전략)를 정의하는 `SamplingParams` 객체 생성부입니다. 모델이 확률 분포(Logits)로부터 다음 토큰을 선택하는 과정을 제어하여, 생성 결과의 품질, 다양성, 그리고 리소스 사용량을 결정짓는 핵심 파라미터들입니다.


### Decoding Strategy Configuration

#### 1. `max_tokens` (Termination Condition)

정의: 모델이 생성할 수 있는 출력 시퀀스의 최대 길이(Maximum Output Length)를 제한하는 Hard Limit입니다. 예를 들어, output 토큰 수가 128에 도달하면 128에서 중간에 생성을 멈춥니다.

* Latency Budgeting: 생성 토큰 수는 추론 시간(Inference Latency)과 정비례합니다. 서비스의 응답 속도 목표(SLA)에 맞춰 적절한 상한선을 설정해야 합니다.
* Resource Management: 생성 길이가 길어질수록 KV Cache 점유량이 증가합니다. 무한 생성을 방지하여 OOM(Out of Memory) 및 연산 리소스 낭비를 막는 안전장치 역할을 합니다.
* Note: 이 값에 도달하면 문장이 완결되지 않았더라도 강제로 생성을 중단(Stop)합니다.



#### 2. `temperature` (Entropy Scaling)

정의: Softmax 함수를 통과하기 전, Logit 값을 스케일링하여 확률 분포의 평탄도(Flatness)를 조절하는 파라미터입니다.

* Low (): 확률 분포를 날카롭게(Peaky) 만듭니다. 확률이 높은 토큰은 더 높아지고, 낮은 토큰은 더 낮아져 결정론적(Deterministic)이고 일관된 결과를 유도합니다. (Code generation 등에 적합)
* High (): 확률 분포를 평평하게(Flatten) 만듭니다. 확률이 낮은 토큰의 선택 가능성을 높여 다양성(Diversity)과 창의성을 부여합니다. (Creative writing 등에 적합)



#### 3. `top_p` (Nucleus Sampling)

정의: 확률이 높은 순서대로 토큰을 정렬한 뒤, 누적 확률(Cumulative Probability)이 값(여기서는 0.9, 즉 90%)에 도달할 때까지만 후보군(Candidate Set)에 포함시키는 동적 절단(Dynamic Truncation) 방식입니다. tokenizer에 있는 vocab에서 생성할 후보들을 고려하게 되는데 90%로 설정하면 나머지 10%의 확률을 차지하는 수많은 토큰들은 고려하지 않게 됩니다. 예를 들어, vocab이 128,000이고, "안녕" 이라는 토큰의 활률이 90% 라고 하면 나머지 127,999의 토큰은 고려하지 않습니다.

* 모델이 확신을 가지는 상황(특정 단어의 확률이 매우 높음)에서는 후보군을 좁게 가져가고,
* 확신이 없는 상황(여러 단어의 확률이 비슷함)에서는 후보군을 넓게 가져갑니다.


* Engineering Insight: `top_k`의 경직성을 보완하여, 문맥에 따라 유동적으로 탐색 공간(Search Space)을 조절함으로써 텍스트의 일관성을 유지하면서도 풍부한 표현을 가능하게 합니다.

#### 4. `top_k` (Top-K Sampling)

정의: 다음 토큰 후보를 확률이 가장 높은 상위 개(여기서는 50개)로 고정하여 제한하는 정적 절단(Static Truncation) 방식입니다. top_p와 유사한 기능이라고 생각하시면 좋습니다.

* Mechanism: 50 번째 이후의 토큰들은 확률이 아무리 높아도(혹은 존재하더라도) 후보군에서 배제(Masking)됩니다.
* 확률 분포의 Long Tail(확률이 매우 낮은 꼬리 부분)을 물리적으로 잘라냅니다.
* 문맥과 전혀 상관없는 엉뚱한 토큰(Outlier)이 선택될 가능성을 원천 차단하여 생성 결과의 안정성을 보장합니다.


In [7]:
# Step 2: Sampling 파라미터 설정
sampling_params = SamplingParams(
    max_tokens=128,        # 최대 생성 토큰 수
    temperature=0.7,       # 샘플링 온도 (높을수록 다양성 증가)
    top_p=0.9,            # Nucleus sampling
    top_k=50,             # Top-k sampling
)

print("⚙️  Sampling 파라미터:")
print(f"  - Max Tokens: {sampling_params.max_tokens}")
print(f"  - Temperature: {sampling_params.temperature}")
print(f"  - Top-p: {sampling_params.top_p}")

⚙️  Sampling 파라미터:
  - Max Tokens: 128
  - Temperature: 0.7
  - Top-p: 0.9


In [8]:
# Step 3: 추론 실행
prompts = [
    "Explain quantum computing in simple terms:",
    "What is the capital of France?",
    "Write a haiku about programming:",
]

print("🤖 추론 시작...")
start_time = time.time()

outputs = llm.generate(prompts, sampling_params)

inference_time = time.time() - start_time

print(f"\n✅ 추론 완료! ({inference_time:.2f}초)")
print(f"📊 처리량: {len(prompts) / inference_time:.2f} prompts/sec\n")

# 결과 출력
for i, output in enumerate(outputs):
    generated_text = output.outputs[0].text
    token_count = len(output.outputs[0].token_ids)
    
    print(f"[프롬프트 {i+1}] {prompts[i]}")
    print(f"[생성 결과] {generated_text}")
    print(f"[토큰 수] {token_count}")
    print("-" * 60)

🤖 추론 시작...


Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


✅ 추론 완료! (2.91초)
📊 처리량: 1.03 prompts/sec

[프롬프트 1] Explain quantum computing in simple terms:
[생성 결과] 

Quantum computing is a type of computing that uses the principles of quantum mechanics to process information. Unlike classical computing, which uses bits to store information, quantum computing uses quantum bits, or qubits. Qubits can exist in multiple states simultaneously, which allows quantum computers to process complex problems much faster than classical computers.

For example, a classical computer can solve a problem in a few minutes, but a quantum computer can solve the same problem in a few seconds. This is because the quantum computer uses quantum entanglement to combine the qubits, allowing the computer to process the problem in a way that classical computers cannot.

Qub
[토큰 수] 128
------------------------------------------------------------
[프롬프트 2] What is the capital of France?
[생성 결과] 
The capital of France is Paris.
#### 3
Is the following statement true?
"The capi

## Part 2.5: KV Cache 성능 비교 시뮬레이터

이제 **왜 KV Cache가 필요한지** 연산 횟수를 카운팅하여 직접 확인해봅시다

이 시뮬레이터는 실제 행렬 연산 대신, '연산을 몇 번 수행했는가'를 측정합니다.

In [9]:
class ModelSimulator:
    """LLM 추론 과정을 시뮬레이션"""
    def __init__(self):
        self.op_count = 0  # 연산 횟수 카운터

    def compute_token_representation(self, token):
        """
        토큰을 벡터로 변환 (Embedding + Q, K, V 생성)
        고비용 연산이라고 가정
        """
        self.op_count += 1
        return f"Vector({token})"

    def attention_calculation(self, current_token, past_vectors):
        """
        현재 토큰과 과거 토큰들 간의 관계 계산 (Attention)
        """
        self.op_count += len(past_vectors)

print("✅ ModelSimulator 준비 완료")

✅ ModelSimulator 준비 완료


In [10]:
# ==========================================
# 시나리오 1: KV Cache 없이 생성 (No Cache)
# ==========================================
print("🚫 [Scenario 1] KV Cache 미사용 (매번 처음부터 다시 계산)")
model_no_cache = ModelSimulator()
generated_tokens = []
input_prompt = ["Deep", "Learning", "is", "Fun"]

for step in range(5):
    current_context = input_prompt + generated_tokens
    
    # 매번 모든 토큰을 다시 계산
    vectors = []
    for token in current_context:
        vec = model_no_cache.compute_token_representation(token)
        vectors.append(vec)
    
    model_no_cache.attention_calculation("New_Token", vectors)
    
    generated_tokens.append(f"Gen_{step}")
    print(f"  Step {step+1}: 문장 길이 {len(current_context)} -> 누적 연산: {model_no_cache.op_count}")

final_cost_no_cache = model_no_cache.op_count
print(f"\n총 연산 횟수: {final_cost_no_cache}")

🚫 [Scenario 1] KV Cache 미사용 (매번 처음부터 다시 계산)
  Step 1: 문장 길이 4 -> 누적 연산: 8
  Step 2: 문장 길이 5 -> 누적 연산: 18
  Step 3: 문장 길이 6 -> 누적 연산: 30
  Step 4: 문장 길이 7 -> 누적 연산: 44
  Step 5: 문장 길이 8 -> 누적 연산: 60

총 연산 횟수: 60


In [11]:
# ==========================================
# 시나리오 2: KV Cache 사용 (With Cache)
# ==========================================
print("\n✅ [Scenario 2] KV Cache 사용 (과거는 저장해두고 재사용)")
model_with_cache = ModelSimulator()
generated_tokens = []
kv_cache = []  # 이것이 바로 KV Cache!

# Prefill: 초기 프롬프트 처리 (한 번만)
print("  [Prefill] 초기 프롬프트 처리 중...")
for token in input_prompt:
    vec = model_with_cache.compute_token_representation(token)
    kv_cache.append(vec)

# Decode: 토큰 생성
for step in range(5):
    # 새로운 토큰 하나만 처리
    current_vec = model_with_cache.compute_token_representation("New_Token")
    
    # 과거 cache + 현재 벡터로 attention
    model_with_cache.attention_calculation("New_Token", kv_cache)
    
    # 캐시에 저장
    kv_cache.append(current_vec)
    generated_tokens.append(f"Gen_{step}")
    
    print(f"  Step {step+1}: 캐시 크기 {len(kv_cache)} -> 누적 연산: {model_with_cache.op_count}")

final_cost_with_cache = model_with_cache.op_count
print(f"\n총 연산 횟수: {final_cost_with_cache}")


✅ [Scenario 2] KV Cache 사용 (과거는 저장해두고 재사용)
  [Prefill] 초기 프롬프트 처리 중...
  Step 1: 캐시 크기 5 -> 누적 연산: 9
  Step 2: 캐시 크기 6 -> 누적 연산: 15
  Step 3: 캐시 크기 7 -> 누적 연산: 22
  Step 4: 캐시 크기 8 -> 누적 연산: 30
  Step 5: 캐시 크기 9 -> 누적 연산: 39

총 연산 횟수: 39


In [12]:
# 최종 비교
print("\n" + "=" * 50)
print("📊 최종 연산 비용 비교 (낮을수록 빠름)")
print("=" * 50)
print(f"1. No Cache Cost  : {final_cost_no_cache} (O(N²) - 기하급수적 증가)")
print(f"2. With Cache Cost: {final_cost_with_cache} (O(N) - 선형적 증가)")
print(f"🚀 효율 향상: 약 {final_cost_no_cache / final_cost_with_cache:.1f}배 더 적은 연산!")
print("=" * 50)


📊 최종 연산 비용 비교 (낮을수록 빠름)
1. No Cache Cost  : 60 (O(N²) - 기하급수적 증가)
2. With Cache Cost: 39 (O(N) - 선형적 증가)
🚀 효율 향상: 약 1.5배 더 적은 연산!


### Performance Benchmark & Analysis (결과 분석)

시뮬레이션 결과는 **KV Cache 메커니즘의 유무**와 **vLLM의 메모리 최적화 전략**이 추론 파이프라인 성능에 미치는 결정적인(Critical) 차이를 보여줍니다.

#### 1. Naive Decoding (No Cache)

* **Computational Complexity**:  (Quadratic)
* **Analysis**: 매 토큰 생성 시점마다 이전의 모든 토큰에 대해 Attention 연산을 처음부터 다시 수행(Re-computation)합니다. 시퀀스 길이(Context Length)가 길어질수록 연산 비용이 기하급수적으로 증가하여, 실시간 서비스가 불가능한 수준의 Latency가 발생합니다.

#### 2. Standard KV Caching

* **Computational Complexity**:  (Linear)
* **Analysis**: 전형적인 **Space-Time Trade-off** 전략입니다. 과거 토큰의 Key/Value State를 VRAM에 상주시켜 중복 연산을 제거했습니다. 속도는 획기적으로 개선되지만, 그 대가로 VRAM 점유율(Footprint)이 급증하여 배치 크기(Batch Size) 확장에 물리적 제약이 생깁니다.

#### 3. vLLM (PagedAttention)

* **Mechanism**: **Zero-Waste Memory Management**
* **Analysis**: KV Cache의 연산 효율성(O(N))은 유지하되, OS 페이징 기법을 통해 **메모리 단편화(Fragmentation)** 문제를 구조적으로 해결했습니다.
* **Impact**: 물리적 메모리의 유휴 공간을 블록 단위로 빈틈없이 활용(Packing)함으로써, 동일한 하드웨어 리소스 내에서 수용 가능한 **KV Cache의 밀도(Density)**를 높입니다. 이는 결과적으로 동시 처리량(Throughput)의 극대화로 직결됩니다.

---
## 3 Chat 형식 프롬프트

실전에서는 시스템 메시지, 유저 메시지, 어시스턴트 메시지를 조합한 chat 형식을 많이 사용합니다.

In [13]:
def format_chat_messages(messages):
    """
    Chat 메시지를 프롬프트로 변환
    
    Args:
        messages: [{'role': 'system/user/assistant', 'content': '...'}, ...]
    """
    if tokenizer.chat_template:
        # Tokenizer에 chat template이 있으면 사용
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        # 수동 포맷팅
        formatted = ""
        for msg in messages:
            role = msg['role'].capitalize()
            content = msg['content']
            formatted += f"{role}: {content}\n\n"
        
        if messages[-1]['role'] != 'assistant':
            formatted += "Assistant: "
        
        return formatted

print("✅ Chat 포맷팅 함수 준비 완료")

✅ Chat 포맷팅 함수 준비 완료


In [14]:
# Text-to-SQL 예제
messages = [
    {
        'role': 'system',
        'content': 'You are a SQL expert. Convert natural language queries to SQL.'
    },
    {
        'role': 'user',
        'content': 'Find all users with age greater than 30'
    }
]

# 프롬프트 변환
prompt = format_chat_messages(messages)
print("📝 변환된 프롬프트:")
print(prompt)
print("-" * 60)

# 추론
sampling_params_sql = SamplingParams(
    max_tokens=64,
    temperature=0.1,  # SQL 생성은 낮은 temperature 사용
)

outputs = llm.generate([prompt], sampling_params_sql)

del llm
print("\n💡 생성된 SQL:")
print(outputs[0].outputs[0].text)

📝 변환된 프롬프트:
<|im_start|>system
You are a SQL expert. Convert natural language queries to SQL.<|im_end|>
<|im_start|>user
Find all users with age greater than 30<|im_end|>
<|im_start|>assistant

------------------------------------------------------------


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[rank0]:[W201 01:00:16.976771580 ProcessGroupNCCL.cpp:1524] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())



💡 생성된 SQL:
SELECT * FROM users WHERE age > 30;


---
## 4 LoRA Adapter 사용하기

### 4.1 LoRA란?

**LoRA (Low-Rank Adaptation)**:
- 모든 파라미터를 fine-tuning하는 대신, 작은 adapter만 학습
- 메모리 효율적이고 빠른 학습 가능
- 여러 task별 adapter를 쉽게 교체 가능

### 4.2 vLLM에서 LoRA 사용하는 두 가지 방법

#### 방법 1: Runtime LoRA (동적 적용)
- 추론 시점에 adapter를 동적으로 로드
- 여러 adapter를 빠르게 교체 가능

skeleton-2에서 학습한 LoRA 모듈을 이용하시거나 skeleton-3에서 학습한 LoRA 모듈을 이용하시면 됩니다.

In [15]:
# Runtime LoRA 예제 (adapter가 있는 경우)
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"
lora_adapter_path = "./checkpoint-100"  # 실제 adapter 경로로 변경

if os.path.exists(lora_adapter_path):
    from vllm.lora.request import LoRARequest
    
    # LoRA 지원 모델 로드
    llm_with_lora = LLM(
        model=model_name,
        enable_lora=True,
        max_lora_rank=64,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.75,
        trust_remote_code=True,
        enforce_eager=True
    )
    
    # LoRA request 생성
    lora_request = LoRARequest("my_adapter", 1, lora_adapter_path) # my_adapter는 임의의 이름. 원하는 이름으로 설정 가능능
    
    # 추론 (LoRA adapter 적용)
    outputs = llm_with_lora.generate(
        [prompt],
        sampling_params,
        lora_request=lora_request
    )
    
    print("✅ LoRA adapter 적용 완료")
    print(outputs)
else:
    print("⚠️ LoRA adapter 없음 - 건너뛰기")

INFO 02-01 01:00:16 [utils.py:253] non-default args: {'trust_remote_code': True, 'gpu_memory_utilization': 0.75, 'disable_log_stats': True, 'enforce_eager': True, 'enable_lora': True, 'max_lora_rank': 64, 'model': 'HuggingFaceTB/SmolLM2-360M-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 02-01 01:00:17 [model.py:514] Resolved architecture: LlamaForCausalLM
INFO 02-01 01:00:17 [model.py:1661] Using max model len 8192
INFO 02-01 01:00:17 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 02-01 01:00:17 [vllm.py:622] Enforce eager set, overriding optimization level to -O0
INFO 02-01 01:00:17 [vllm.py:722] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:21 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='HuggingFaceTB/SmolLM2-360M-Instruct', speculative_config=None, tokenizer='HuggingFaceTB/SmolLM2-360M-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, device_config=

(EngineCore_DP0 pid=5103) /home/ssafy/skeleton-4/.venv/lib/python3.13/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=5103) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=5103)   warnings.warn(


(EngineCore_DP0 pid=5103) INFO 02-01 01:00:25 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:26 [weight_utils.py:527] No model.safetensors.index.json found in remote.


...

(EngineCore_DP0 pid=5103) INFO 02-01 01:00:26 [default_loader.py:308] Loading weights took 0.27 seconds
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:26 [punica_selector.py:20] Using PunicaWrapperGPU.
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:27 [gpu_model_runner.py:3659] Model loading took 0.7521 GiB memory and 3.339144 seconds
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:28 [gpu_worker.py:375] Available KV cache memory: 3.25 GiB
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:29 [kv_cache_utils.py:1291] GPU KV cache size: 85,168 tokens
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:29 [kv_cache_utils.py:1296] Maximum concurrency for 8,192 tokens per request: 10.40x
(EngineCore_DP0 pid=5103) INFO 02-01 01:00:29 [core.py:259] init engine (profile, create kv cache, warmup model) took 1.68 seconds
(EngineCore_DP0 pid=5103) WARNING 02-01 01:00:30 [vllm.py:629] Inductor compilation was disabled by user settings,Optimizations settings that are only active duringInductor compilation will be ignored.
(Eng

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

WARNING 02-01 01:00:30 [input_processor.py:250] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

(EngineCore_DP0 pid=5103) WARNING 02-01 01:00:30 [utils.py:250] Using default LoRA kernel configs
✅ LoRA adapter 적용 완료
[RequestOutput(request_id=0, prompt='<|im_start|>system\nYou are a SQL expert. Convert natural language queries to SQL.<|im_end|>\n<|im_start|>user\nFind all users with age greater than 30<|im_end|>\n<|im_start|>assistant\n', prompt_token_ids=[1, 9690, 198, 2683, 359, 253, 15142, 4507, 30, 29490, 1782, 1789, 18795, 288, 15142, 30, 2, 198, 1, 4093, 198, 11933, 511, 3629, 351, 1850, 2852, 670, 216, 35, 32, 2, 198, 1, 520, 9531, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='SELECT name \nFROM users \nWHERE age > 30', token_ids=[23428, 1462, 216, 198, 46510, 3629, 216, 198, 14779, 14298, 1850, 2986, 216, 35, 32, 2], cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


#### 방법 2: Merged Model (사전 통합)
- LoRA weights를 base model에 미리 merge
- 더 빠른 추론 속도
- 배포에 유리

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel



def merge_lora_to_base(base_model_name, lora_path, output_path):
    """
    LoRA adapter를 base model에 merge
    """
    print("🔄 LoRA merge 시작...")
    
    # 1. Base model 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        trust_remote_code=True,
        torch_dtype="auto",
    )
    
    # 2. LoRA adapter 로드
    model_with_lora = PeftModel.from_pretrained(base_model, lora_path)
    
    # 3. Merge
    merged_model = model_with_lora.merge_and_unload()
    
    # 4. 저장
    os.makedirs(output_path, exist_ok=True)
    merged_model.save_pretrained(output_path, safe_serialization=True)
    
    # Tokenizer도 저장
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.save_pretrained(output_path)

    del base_model
    del model_with_lora
    del merged_model
    
    print(f"✅ Merge 완료: {output_path}")
    return output_path

# 예제 (adapter가 있는 경우)
if os.path.exists(lora_adapter_path):
    if locals().get('llm_with_lora'):
        del llm_with_lora

    merged_path = merge_lora_to_base(
        base_model_name=model_name,
        lora_path=lora_adapter_path,
        output_path="./merged_model"
    )
    
    # Merged 모델로 추론
    llm_merged = LLM(
        model=merged_path,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.75,
        trust_remote_code=True,
    )
    print("✅ Merged 모델로 추론 준비 완료")
else:
    print("⚠️ LoRA adapter 없음 - 건너뛰기")

[rank0]:[W201 01:00:32.436326660 ProcessGroupNCCL.cpp:1524] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


🔄 LoRA merge 시작...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Merge 완료: ./merged_model
INFO 02-01 01:00:36 [utils.py:253] non-default args: {'trust_remote_code': True, 'gpu_memory_utilization': 0.75, 'disable_log_stats': True, 'model': './merged_model'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


...

(EngineCore_DP0 pid=5294) INFO 02-01 01:00:44 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.97it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.97it/s]
(EngineCore_DP0 pid=5294) 


(EngineCore_DP0 pid=5294) INFO 02-01 01:00:44 [default_loader.py:308] Loading weights took 0.18 seconds
(EngineCore_DP0 pid=5294) INFO 02-01 01:00:45 [gpu_model_runner.py:3659] Model loading took 0.6750 GiB memory and 1.939831 seconds
(EngineCore_DP0 pid=5294) INFO 02-01 01:00:50 [backends.py:643] Using cache directory: /home/ssafy/.cache/vllm/torch_compile_cache/45e4fc4647/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=5294) INFO 02-01 01:00:50 [backends.py:703] Dynamo bytecode transform time: 4.36 s
(EngineCore_DP0 pid=5294) INFO 02-01 01:00:56 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 1.022 s
(EngineCore_DP0 pid=5294) INFO 02-01 01:00:56 [monitor.py:34] torch.compile takes 5.39 s in total
(EngineCore_DP0 pid=5294) INFO 02-01 01:00:57 [gpu_worker.py:375] Available KV cache memory: 3.33 GiB
(EngineCore_DP0 pid=5294) INFO 02-01 01:00:57 [kv_cache_utils.py:1291] GPU KV cache size: 87,168 tokens
(EngineCore_DP0 

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:02<00:00, 17.58it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:02<00:00, 14.69it/s]


(EngineCore_DP0 pid=5294) INFO 02-01 01:01:04 [gpu_model_runner.py:4587] Graph capturing finished in 7 secs, took 0.49 GiB
(EngineCore_DP0 pid=5294) INFO 02-01 01:01:04 [core.py:259] init engine (profile, create kv cache, warmup model) took 18.75 seconds
INFO 02-01 01:01:04 [llm.py:360] Supported tasks: ['generate']
✅ Merged 모델로 추론 준비 완료


---
## Part 5: 성능 벤치마크 - Transformers vs vLLM

vLLM이 얼마나 빠른지 직접 측정해봅니다.

In [17]:
# 벤치마크 준비
test_prompts = [
    "You are a SQL expert. Convert this to SQL: Find all users",
    "You are a SQL expert. Convert this to SQL: Count employees",
    "You are a SQL expert. Convert this to SQL: Show top 10 sales",
    "You are a SQL expert. Convert this to SQL: Delete inactive accounts",
    "You are a SQL expert. Convert this to SQL: Update user emails",
]

max_tokens = 64

print(f"📊 벤치마크 설정")
print(f"  - 프롬프트 수: {len(test_prompts)}")
print(f"  - Max Tokens: {max_tokens}")

📊 벤치마크 설정
  - 프롬프트 수: 5
  - Max Tokens: 64


### 5.1 Transformers 벤치마크

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pynvml

def get_gpu_memory_usage():
    """NVIDIA GPU에서 실제 메모리 사용량을 측정 (nvidia-smi와 동일)"""
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    pynvml.nvmlShutdown()
    return info.used / 1024 / 1024  # MB 단위

print("🔵 Transformers 벤치마크 시작")
print("  모델 로딩 중...")

# 모델 로드
tf_tokenizer = AutoTokenizer.from_pretrained(model_name)
tf_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
tf_model.eval()

# 추론
tf_start = time.time()
tf_total_tokens = 0
tf_first_token_latencies = []

for prompt in test_prompts:
    inputs = tf_tokenizer(prompt, return_tensors="pt").to(tf_model.device)
    
    # First token 측정
    ft_start = time.time()
    with torch.no_grad():
        outputs = tf_model(**inputs)
    first_token_time = time.time() - ft_start
    tf_first_token_latencies.append(first_token_time)
    
    # 전체 생성
    with torch.no_grad():
        generated = tf_model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
        )
    tf_total_tokens += generated.shape[1] - inputs.input_ids.shape[1]

tf_time = time.time() - tf_start
tf_peak_memory = get_gpu_memory_usage()  # NVIDIA GPU 실제 메모리 측정
tf_avg_first_token = sum(tf_first_token_latencies) / len(tf_first_token_latencies)

print(f"\n✅ Transformers 완료")
print(f"  ⏱️  총 시간: {tf_time:.2f}s")
print(f"  ⚡ First Token: {tf_avg_first_token*1000:.2f}ms")
print(f"  🔥 Token/sec: {tf_total_tokens / tf_time:.2f}")
print(f"  💾 피크 메모리: {tf_peak_memory:.0f} MB")

# 메모리 정리
del tf_model
del tf_tokenizer
if torch.cuda.is_available():
    torch.cuda.empty_cache()

🔵 Transformers 벤치마크 시작
  모델 로딩 중...


`torch_dtype` is deprecated! Use `dtype` instead!
[2026-02-01 01:01:05] INFO modeling.py:1004: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).



✅ Transformers 완료
  ⏱️  총 시간: 5.02s
  ⚡ First Token: 98.86ms
  🔥 Token/sec: 23.30
  💾 피크 메모리: 6134 MB


### 5.2 vLLM 벤치마크

In [19]:
print("\n🟢 vLLM 벤치마크 시작")

# Sampling 파라미터
benchmark_params = SamplingParams(
    max_tokens=max_tokens,
    temperature=0.0,  # greedy decoding
)

# First token latency 측정
vllm_ft_start = time.time()
_ = llm_merged.generate([test_prompts[0]], benchmark_params)
vllm_first_token = time.time() - vllm_ft_start

# 전체 추론
vllm_start = time.time()
vllm_outputs = llm_merged.generate(test_prompts, benchmark_params)
vllm_time = time.time() - vllm_start

vllm_peak_memory = get_gpu_memory_usage()  # NVIDIA GPU 실제 메모리 측정
vllm_total_tokens = sum(len(out.outputs[0].token_ids) for out in vllm_outputs)

print(f"\n✅ vLLM 완료")
print(f"  ⏱️  총 시간: {vllm_time:.2f}s")
print(f"  ⚡ First Token: {vllm_first_token*1000:.2f}ms")
print(f"  🔥 Token/sec: {vllm_total_tokens / vllm_time:.2f}")
print(f"  💾 피크 메모리: {vllm_peak_memory:.0f} MB")


🟢 vLLM 벤치마크 시작


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


✅ vLLM 완료
  ⏱️  총 시간: 0.42s
  ⚡ First Token: 137.75ms
  🔥 Token/sec: 212.99
  💾 피크 메모리: 6022 MB


### 5.3 결과 비교

In [20]:
print("\n" + "=" * 60)
print("📊 최종 비교 결과")
print("=" * 60)

# First Token Latency
print(f"\n⚡ First Token Latency:")
print(f"  Transformers: {tf_avg_first_token*1000:.2f}ms")
print(f"  vLLM:         {vllm_first_token*1000:.2f}ms")
ft_improvement = ((tf_avg_first_token - vllm_first_token) / tf_avg_first_token) * 100
print(f"  {'🚀 개선' if ft_improvement > 0 else '⚠️ 차이'}: {abs(ft_improvement):.1f}%")

# Token/sec
tf_tps = tf_total_tokens / tf_time
vllm_tps = vllm_total_tokens / vllm_time
print(f"\n🔥 Token/sec:")
print(f"  Transformers: {tf_tps:.2f} tokens/sec")
print(f"  vLLM:         {vllm_tps:.2f} tokens/sec")
print(f"  🚀 vLLM 향상: {vllm_tps / tf_tps:.2f}x")

# 총 시간
print(f"\n⏱️ 총 추론 시간:")
print(f"  Transformers: {tf_time:.2f}s")
print(f"  vLLM:         {vllm_time:.2f}s")
print(f"  🚀 속도 향상: {tf_time / vllm_time:.2f}x")

# 메모리
print(f"\n💾 피크 GPU 메모리:")
print(f"  Transformers: {tf_peak_memory:.0f} MB")
print(f"  vLLM:         {vllm_peak_memory:.0f} MB")
memory_diff = tf_peak_memory - vllm_peak_memory
if memory_diff > 0:
    print(f"  💡 메모리 절감: {memory_diff:.0f} MB ({memory_diff/tf_peak_memory*100:.1f}%)")
else:
    print(f"  📈 메모리 사용: {abs(memory_diff):.0f} MB 더 사용")

print("\n" + "=" * 60)


📊 최종 비교 결과

⚡ First Token Latency:
  Transformers: 98.86ms
  vLLM:         137.75ms
  ⚠️ 차이: 39.3%

🔥 Token/sec:
  Transformers: 23.30 tokens/sec
  vLLM:         212.99 tokens/sec
  🚀 vLLM 향상: 9.14x

⏱️ 총 추론 시간:
  Transformers: 5.02s
  vLLM:         0.42s
  🚀 속도 향상: 11.88x

💾 피크 GPU 메모리:
  Transformers: 6134 MB
  vLLM:         6022 MB
  💡 메모리 절감: 112 MB (1.8%)



## 6: 종합 실습

### 최종 프로젝트: Text-to-SQL 시스템 구축

배운 내용을 종합하여 실전 Text-to-SQL 시스템을 만들어봅시다.

In [21]:
system_prompt = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query."""
user_prompt = """Given the <USER_QUERY>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<USER_QUERY>
{question}
</USER_QUERY>"""

test_questions = [
    "How many different types of pet are there?",
    "Count employees in each department",
    "How many airlines do we have?",
    "List departments with budget over 100000",
]

messages = []
for i in range(len(test_questions)):
    messages.append(
        [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt.format(question=test_questions[i])
            }
        ]
    )

prompts = format_chat_messages(messages)

print("🤖 추론 시작...")
start_time = time.time()

outputs = llm_merged.generate(prompts, sampling_params)

inference_time = time.time() - start_time

print(f"\n✅ 추론 완료! ({inference_time:.2f}초)")
print(f"📊 처리량: {len(prompts) / inference_time:.2f} prompts/sec\n")

# 결과 출력
for i, output in enumerate(outputs):
    generated_text = output.outputs[0].text
    token_count = len(output.outputs[0].token_ids)
    
    print(f"[생성 결과] {generated_text}")
    print(f"[토큰 수] {token_count}")
    print("-" * 60)

🤖 추론 시작...


Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


✅ 추론 완료! (0.19초)
📊 처리량: 21.51 prompts/sec

[생성 결과] SELECT COUNT(*) FROM pets;
[토큰 수] 9
------------------------------------------------------------
[생성 결과] SELECT COUNT(*) FROM employee WHERE department = 'Department A'
[토큰 수] 15
------------------------------------------------------------
[생성 결과] SELECT COUNT(*) FROM airlines
[토큰 수] 8
------------------------------------------------------------
[생성 결과] SELECT * FROM `department` WHERE `budget` > 100000
[토큰 수] 19
------------------------------------------------------------
